In [ ]:
import os
import joblib
import random
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping
from keras.layers import Input, LSTM, Dense
from keras.models import Model
from keras.utils import to_categorical
from keras import Input

In [ ]:
training_df = pd.read_csv("/content/drive/MyDrive/Video Captioning/Data/training_captions.csv")
training_df.head()

,ID,captions
0,-4wsuPCjDBc_5_15,a squirrel is eating a peanut in it s shell\n
1,-4wsuPCjDBc_5_15,a chipmunk is eating\n
2,-4wsuPCjDBc_5_15,a chipmunk is eating a peanut\n
3,-4wsuPCjDBc_5_15,a chipmunk is eating a nut\n
4,-4wsuPCjDBc_5_15,a squirrel is eating a nut\n


In [ ]:
files = os.listdir("/content/drive/MyDrive/Video Captioning/Data/Training Videos/feat")
len(files)

714

In [ ]:
train_map = {vid[:-8]: idx for idx, vid in enumerate(files)}
len(train_map)

714

In [ ]:
training_df['captions'] = training_df['captions'].str[:-1]
training_df.head()

,ID,captions
0,-4wsuPCjDBc_5_15,a squirrel is eating a peanut in it s shell
1,-4wsuPCjDBc_5_15,a chipmunk is eating
2,-4wsuPCjDBc_5_15,a chipmunk is eating a peanut
3,-4wsuPCjDBc_5_15,a chipmunk is eating a nut
4,-4wsuPCjDBc_5_15,a squirrel is eating a nut


In [ ]:
training_df = training_df[pd.notnull(training_df['captions'])]
training_df.head()

,ID,captions
0,-4wsuPCjDBc_5_15,a squirrel is eating a peanut in it s shell
1,-4wsuPCjDBc_5_15,a chipmunk is eating
2,-4wsuPCjDBc_5_15,a chipmunk is eating a peanut
3,-4wsuPCjDBc_5_15,a chipmunk is eating a nut
4,-4wsuPCjDBc_5_15,a squirrel is eating a nut


In [ ]:
training_df.describe()

,ID,captions
count,76779,76779
unique,1832,58885
top,#NAME?,a man is playing a guitar
freq,1611,319


In [ ]:
for i in training_df.index:
  if training_df['ID'][i] not in train_map:
    training_df.drop(i, inplace=True)
training_df.describe()

,ID,captions
count,28711,28711
unique,698,22866
top,kWLNZzuo3do_48_53,a man is playing a guitar
freq,81,151


In [ ]:
training_df['map'] = ""
for i in training_df.index:
  training_df['map'][i] = train_map[training_df['ID'][i]]
training_df.head()

,ID,captions,map
0,-4wsuPCjDBc_5_15,a squirrel is eating a peanut in it s shell,283
1,-4wsuPCjDBc_5_15,a chipmunk is eating,283
2,-4wsuPCjDBc_5_15,a chipmunk is eating a peanut,283
3,-4wsuPCjDBc_5_15,a chipmunk is eating a nut,283
4,-4wsuPCjDBc_5_15,a squirrel is eating a nut,283


In [ ]:
training_df.to_csv("/content/drive/MyDrive/Video Captioning/Data/Cleaned_training_captions.csv")

In [ ]:
len(training_df)

28711

In [ ]:
training_df.sort_values(by=['map'], inplace=True)
training_df.head()

,ID,captions,map
9615,5fq9iIxxA1s_12_27,this is airtel advertisement,0
9629,5fq9iIxxA1s_12_27,a small boy is encouraging another to kick a f...,0
9630,5fq9iIxxA1s_12_27,a little boy is kicking a soccer ball,0
9631,5fq9iIxxA1s_12_27,a boy appears to be trying to get another boy ...,0
9632,5fq9iIxxA1s_12_27,a boy signals another young boy to kick a ball,0


In [ ]:
count = training_df['captions'].str.split().apply(len).value_counts()
count.index = count.index.astype(int)
count.sort_index(inplace=True)
count

1       21
2      153
3      776
4     2946
5     4412
6     6372
7     4493
8     3403
9     2191
10    1374
11     842
12     515
13     352
14     222
15     160
16     108
17      77
18      72
19      48
20      46
21      37
22      18
23      19
24      19
25      10
26       6
27       3
28       1
29       3
30       1
33       2
38       2
40       1
41       2
42       1
43       1
45       2
Name: captions, dtype: int64

In [ ]:
total = 0
for x in count.index:
  if x<=10 and x>=4:
    total += count[x]
total

25191

In [ ]:
training_df.reset_index(inplace = True)

In [ ]:
training_df.head()

,index,ID,captions,map
0,9615,5fq9iIxxA1s_12_27,this is airtel advertisement,0
1,9629,5fq9iIxxA1s_12_27,a small boy is encouraging another to kick a f...,0
2,9630,5fq9iIxxA1s_12_27,a little boy is kicking a soccer ball,0
3,9631,5fq9iIxxA1s_12_27,a boy appears to be trying to get another boy ...,0
4,9632,5fq9iIxxA1s_12_27,a boy signals another young boy to kick a ball,0


In [ ]:
training_df = pd.read_csv("/content/drive/MyDrive/Video Captioning/Data/Cleaned_training_captions.csv")
training_df.head()

,Unnamed: 0,ID,captions,map
0,0,-4wsuPCjDBc_5_15,a squirrel is eating a peanut in it s shell,283
1,1,-4wsuPCjDBc_5_15,a chipmunk is eating,283
2,2,-4wsuPCjDBc_5_15,a chipmunk is eating a peanut,283
3,3,-4wsuPCjDBc_5_15,a chipmunk is eating a nut,283
4,4,-4wsuPCjDBc_5_15,a squirrel is eating a nut,283


In [ ]:
training_df['captions'][0]

'a squirrel is eating a peanut in it s shell'

In [ ]:
count = training_df['captions'].str.split().apply(len).value_counts()
count.index = count.index.astype(int)
count.sort_index(inplace=True)
count

1       21
2      153
3      776
4     2946
5     4412
6     6372
7     4493
8     3403
9     2191
10    1374
11     842
12     515
13     352
14     222
15     160
16     108
17      77
18      72
19      48
20      46
21      37
22      18
23      19
24      19
25      10
26       6
27       3
28       1
29       3
30       1
33       2
38       2
40       1
41       2
42       1
43       1
45       2
Name: captions, dtype: int64

In [ ]:
total = 0
for x in count.index:
  if x<=10 and x>=4:
    total += count[x]
total

25191